# NetNTLM Downgrade Attack
Detects post exploitation using NetNTLM downgrade attacks

## Rule Content
```
- action: global
  title: NetNTLM Downgrade Attack
  id: d67572a0-e2ec-45d6-b8db-c100d14b8ef2
  description: Detects post exploitation using NetNTLM downgrade attacks
  references:
  - https://www.optiv.com/blog/post-exploitation-using-netntlm-downgrade-attacks
  author: Florian Roth
  date: 2018/03/20
  tags:
  - attack.credential_access
  - attack.t1212
  detection:
    condition: 1 of them
  falsepositives:
  - Unknown
  level: critical
- logsource:
    product: windows
    service: sysmon
  detection:
    selection1:
      EventID: 13
      TargetObject:
      - '*SYSTEM\\*ControlSet*\Control\Lsa\lmcompatibilitylevel'
      - '*SYSTEM\\*ControlSet*\Control\Lsa\NtlmMinClientSec'
      - '*SYSTEM\\*ControlSet*\Control\Lsa\RestrictSendingNTLMTraffic'
- logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : Object Access > Audit Registry (Success)'
  detection:
    selection2:
      EventID: 4657
      ObjectName: \REGISTRY\MACHINE\SYSTEM\\*ControlSet*\Control\Lsa
      ObjectValueName:
      - LmCompatibilityLevel
      - NtlmMinClientSec
      - RestrictSendingNTLMTraffic

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*SYSTEM\\*ControlSet*\\Control\\Lsa\\lmcompatibilitylevel OR *SYSTEM\\*ControlSet*\\Control\\Lsa\\NtlmMinClientSec OR *SYSTEM\\*ControlSet*\\Control\\Lsa\\RestrictSendingNTLMTraffic))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4657" AND object_name.keyword:\\REGISTRY\\MACHINE\\SYSTEM\\*ControlSet*\\Control\\Lsa AND object_value_name:("LmCompatibilityLevel" OR "NtlmMinClientSec" OR "RestrictSendingNTLMTraffic"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()